In [ ]:
import torch # type: ignore
import torch.nn as nn # type: ignore
from torchvision import datasets, models, transforms # type: ignore
from torch.utils.data import DataLoader, Dataset # type: ignore
from torch.optim import Adam, lr_scheduler # type: ignore
from torchsummary import summary # type: ignore
import matplotlib.pyplot as plt # type: ignore
import numpy as np # type: ignore
from imgaug import augmenters as iaa # type: ignore
import cv2 # type: ignore

dataset_folder = r"C:\Users\jayme\OneDrive\Documents\AI section\Datasets"

augmenter = iaa.Sequential([
    iaa.Multiply(1.2)
], random_order=True)

cifar_train = datasets.CIFAR10(dataset_folder, download=True, train=True)
cifar_test = datasets.CIFAR10(dataset_folder, download=True, train=False)

train_images, train_labels = cifar_train.data, cifar_train.targets
val_images, val_labels = cifar_test.data, cifar_test.targets

class Data(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.aug = augmenter
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        img = self.x[ix]
        label = self.y[ix]
        img = cv2.resize(img, (224, 224))
        img = self.aug.augment_image(img)
        img = torch.tensor(img / 255)
        img = img.permute(2, 0, 1)
        img = self.normalize(img)
        return img.float(), torch.tensor(label).long()

def Dataloaders():
    train_data = Data(train_images, train_labels)
    test_data = Data(val_images, val_labels)
    train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
    val_dl = DataLoader(test_data, batch_size=32, shuffle=True)
    return train_data, test_data, train_dl, val_dl

train_data, test_data, train_dl, val_dl = Dataloaders()

def Compile():
    model = models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    model.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512 * 1 * 1, 128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, 10),
        nn.LogSoftmax(dim=1)
    )
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(model.classifier.parameters(), lr=1e-3) 
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                               factor=0.5,
                                               patience=2,
                                               min_lr=1e-5,
                                               verbose=True,
                                               threshold=1e-4,
                                               threshold_mode="abs")
    
    return model, loss_func, optimizer, scheduler

model, loss_func, optimizer, scheduler = Compile()

def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    is_correct = (prediction > 0.5) == y
    return is_correct.numpy()

def train_model(model, train_dl, val_dl, loss_func, optimizer, scheduler, epochs=10):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
        model.train()
        train_loss, correct_train = 0, 0
        for images, labels in train_dl:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            correct_train += (outputs.argmax(dim=1) == labels).sum().item()
        
        train_losses.append(train_loss / len(train_dl))
        train_accuracy = correct_train / len(train_dl.dataset)
        train_accuracies.append(train_accuracy)

        model.eval()
        val_loss, correct_val = 0, 0
        with torch.no_grad():
            for images, labels in val_dl:
                outputs = model(images)
                loss = loss_func(outputs, labels)
                val_loss += loss.item()
                correct_val += (outputs.argmax(dim=1) == labels).sum().item()

        val_losses.append(val_loss / len(val_dl))
        val_accuracy = correct_val / len(val_dl.dataset)
        val_accuracies.append(val_accuracy)

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Loss: {train_loss / len(train_dl):.4f}, "
              f"Val Loss: {val_loss / len(val_dl):.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Val Accuracy: {val_accuracy:.4f}")

    return train_losses, val_losses, train_accuracies, val_accuracies


def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Over Epochs')

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy Over Epochs')

    plt.show()


epochs = 10
train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model, train_dl, val_dl, loss_func, optimizer, scheduler, epochs=epochs)

plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies)

def predict_image(model, image_path):
    model.eval()
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = torch.tensor(img / 255).permute(2, 0, 1).unsqueeze(0)
    img = img.float()
    with torch.no_grad():
        output = model(img)
        predicted_class = output.argmax(dim=1).item()
    return predicted_class

image_path = "path_to_your_image.jpg" 
predicted_class = predict_image(model, image_path)
print(f"Predicted Class: {predicted_class}")

In [62]:
#                ======================= CNN With Tensorfow =======================




In [61]:
#                   ======================== PyTorch Project ========================


